In [2]:
import os, sys
import numpy as np
import cv2
import dlib
from google.colab.patches import cv2_imshow
import scipy.spatial as spatial
from scipy.spatial.distance import euclidean as euclid_distance
%matplotlib inline
from matplotlib import pyplot as plt

# upload pictures
!wget -q https://github.com/uol-mediaprocessing/group-projects-group-photo/raw/master/pictures/group_2/index.txt -O group_2.txt
!mkdir group_2

!xargs -i -a group_2.txt wget -q https://github.com/uol-mediaprocessing/group-projects-group-photo/raw/master/pictures/group_2/{} -O group_2/{}
!ls -lah group_2

path = './group_2'

files = os.listdir(path)
files = list(map(lambda f: path + '/' + f, files))
print(files)

imgs = list(map(cv2.imread, files))

# get models
!wget -q  https://github.com/uol-mediaprocessing/group-projects-group-photo/raw/master/shape_predictor_68_face_landmarks.dat.bz2 -O shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
!wget -q https://github.com/uol-mediaprocessing/group-projects-group-photo/raw/master/dlib_face_recognition_resnet_model_v1.dat.bz2
!bzip2 -d dlib_face_recognition_resnet_model_v1.dat.bz2
!wget https://raw.githubusercontent.com/opencv/opencv/3.4/data/haarcascades/haarcascade_smile.xml

total 40M
drwxr-xr-x 2 root root 4.0K Jan 23 11:01 .
drwxr-xr-x 1 root root 4.0K Jan 23 11:00 ..
-rw-r--r-- 1 root root 3.7M Jan 23 11:00 IMG_20191112_111241.jpg
-rw-r--r-- 1 root root 3.6M Jan 23 11:00 IMG_20191112_111244.jpg
-rw-r--r-- 1 root root 4.0M Jan 23 11:00 IMG_20191112_111256.jpg
-rw-r--r-- 1 root root 3.9M Jan 23 11:01 IMG_20191112_111257.jpg
-rw-r--r-- 1 root root 3.8M Jan 23 11:01 IMG_20191112_111300.jpg
-rw-r--r-- 1 root root 4.2M Jan 23 11:01 IMG_20191112_111301.jpg
-rw-r--r-- 1 root root 3.8M Jan 23 11:01 IMG_20191112_111313.jpg
-rw-r--r-- 1 root root 3.3M Jan 23 11:01 IMG_20191112_111315.jpg
-rw-r--r-- 1 root root 3.4M Jan 23 11:01 IMG_20191112_111327.jpg
-rw-r--r-- 1 root root 3.5M Jan 23 11:01 IMG_20191112_111330.jpg
-rw-r--r-- 1 root root 3.3M Jan 23 11:01 IMG_20191112_111332.jpg
['./group_2/IMG_20191112_111327.jpg', './group_2/IMG_20191112_111313.jpg', './group_2/IMG_20191112_111300.jpg', './group_2/IMG_20191112_111244.jpg', './group_2/IMG_20191112_111257.jpg', '.

In [0]:
import math

class Person:
  def __init__(self, id):
    self.id = id
    self.faces = []

  def faceVectorMatches(self, vector):
    return euclid_distance(self.id,vector) < 0.6

  def normalize_scores(self):
    for score_name in ["orientation", "eyes", "mouth"]:
      attr_name = "score_" + score_name
      max = float("-inf")
      for face in self.faces:
        score = getattr(face, attr_name)
        if score > max:
          max = score
      for face in self.faces:
        if max == 0: break
        score = getattr(face, attr_name) / max
        setattr(face, attr_name, score)
      for face in self.faces:
        face.calc_score_all()

  def getFaceFromPic(self,idx):
    return next((face for face in self.faces if face.picture_index == idx),None)
      

class Face:
  def __init__(self, picture_data, picture_index, landmarks,coords):
    self.picture_data = picture_data
    self.picture_index = picture_index
    self.mask = None
    self.convexHull = None
    self.triangles = None
    self.landmarks = landmarks
    self.score_orientation = None
    self.score_eyes = None
    self.score_mouth = None
    self.score_all = None
    self.coords = coords

  def landmarksToFullSize(self):
    landmarksList = []
    for x in shape_to_np(landmarks):
      landmarksList.append((x[0]+self.coords.left(),x[1]+self.coords.top()))
    return np.array(landmarksList)
    
  def show(self):
    cv2_imshow(self.picture_data)

  def calc_face_mask(self):
    height, width, _ = self.picture_data.shape
    mask = np.zeros((height,width,1),dtype= np.uint8)
    landmark_points = shape_to_np(self.landmarks)
    self.convexHull = cv2.convexHull(landmark_points)
    #cv2.polylines(img, [convexhull], True, (255, 0, 0), 3)
    cv2.fillConvexPoly(mask, self.convexHull, (255,255,255))
    self.mask = cv2.bitwise_and(self.picture_data, self.picture_data, mask=mask)

  def delauney_triangulation(self):
    rect = cv2.boundingRect(self.convexHull)
    subdiv = cv2.Subdiv2D(rect)
    landmark_points = shape_to_list(self.landmarks)
    landmark_points = landmark_points[:22]
    subdiv.insert(landmark_points)
    triangles = subdiv.getTriangleList()
    self.triangles = np.array(triangles, dtype=np.int32)
    img_copy = cv2.cvtColor(self.picture_data,cv2.COLOR_RGB2BGR).copy()
    for triangle in triangles:
      pt1 = (triangle[0],triangle[1])
      pt2 = (triangle[2],triangle[3])
      pt3 = (triangle[4],triangle[5])
      cv2.line(img_copy,pt1,pt2,(0,0,255),1)
      cv2.line(img_copy,pt2,pt3,(0,0,255),1)
      cv2.line(img_copy,pt1,pt3,(0,0,255),1)
    cv2_imshow(img_copy)
    
  def calc_score_eyes(self):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    landmarks = shape_to_np(self.landmarks)
    left_eye = landmarks[36:42]
    right_eye = landmarks[42:48]
    eyes = [left_eye,right_eye]
    eyes_score = []
    
    for eye in eyes:
      A = euclid_distance(eye[1], eye[5])
      B = euclid_distance(eye[2], eye[4])

      # compute the euclidean distance between the horizontal
      # eye landmark (x, y)-coordinates
      C = euclid_distance(eye[0], eye[3])

      # compute the eye aspect ratio
      ear = (A + B) / (2.0 * C)
      # return the eye aspect ratio
      eyes_score.append(ear)

    self.score_eyes = (eyes_score[0]+eyes_score[1])/2.0


  def calc_score_orientation(self):
    nparray = shape_to_np(landmarks)
    #red = (255, 0, 0)
    #for x in range(len(nparray)):
      #cv2.circle(picture, (nparray[x][0],nparray[x][1]), 3, red, -1)
    # 2, 30 , 14
    dist1 = calculateDistance(nparray[2][0],nparray[2][1],nparray[30][0],nparray[30][1])
    dist2 = calculateDistance(nparray[14][0],nparray[14][1],nparray[30][0],nparray[30][1])
    distRatio = abs(dist2 - dist1)

    self.score_orientation = 1 / distRatio

  def calc_score_mouth(self):
    gray = cv2.cvtColor(cv2.cvtColor(self.picture_data,cv2.COLOR_RGB2BGR),cv2.COLOR_BGR2GRAY)
    smile = face_cascade_smile.detectMultiScale3(
        gray,
        scaleFactor=1.8,
        minNeighbors=20,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE,
        outputRejectLevels = True
    )
    rects = smile[0]
    weights = smile[2]
    if len(weights) == 1:
      self.score_mouth = float(weights[0])
      #print(float(weights[0]))
    else:
      self.score_mouth = 0

  def calc_scores(self):
    self.calc_score_orientation()
    self.calc_score_eyes()
    self.calc_score_mouth()

    self.calc_score_all()
    
  # sets and return score_all, parameters are for weighting the scores
  def calc_score_all(self, w1 = 10, w2 = 2, w3 = 4):
    self.score_all = self.score_orientation * w1 + self.score_eyes * w2 + self.score_mouth * w3

  def scores_str(self):
    return str({
        "a": round(self.score_all, 3),
        "o": round(self.score_orientation, 3),
        "e": round(self.score_eyes, 3),
        "m": round(self.score_mouth, 3)
    }).replace(",", ",\n")

def applyAffineTransform(src, srcTri, dstTri, size) :
  
  # Given a pair of triangles, find the affine transform.
  warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
  
  # Apply the Affine Transform just found to the src image
  dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

  return dst

# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, t1, t2) :

  # Find bounding rectangle for each triangle
  r1 = cv2.boundingRect(np.float32([t1]))
  r2 = cv2.boundingRect(np.float32([t2]))

  # Offset points by left top corner of the respective rectangles
  t1Rect = [] 
  t2Rect = []
  t2RectInt = []

  for i in range(0, 3):
      t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
      t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
      t2RectInt.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


  # Get mask by filling triangle
  mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
  cv2.fillConvexPoly(mask, np.int32(t2RectInt), (1.0, 1.0, 1.0), 16, 0);

  # Apply warpImage to small rectangular patches
  img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
  #img2Rect = np.zeros((r2[3], r2[2]), dtype = img1Rect.dtype)

  size = (r2[2], r2[3])

  img2Rect = applyAffineTransform(img1Rect, t1Rect, t2Rect, size)
  #cv2_imshow(img2Rect)
  img2Rect = img2Rect * mask
  #cv2_imshow(img2Rect)
  # Copy triangular region of the rectangular patch to the output image
  img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask )
  #cv2_imshow(img2)
  img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Rect 
  #cv2_imshow(img2)

# return the face given a picture and the coords of the face
def extractFace(pic, coords, r = 30):
  return pic[coords.top()-r:coords.bottom()+r, coords.left()-r:coords.right()+r]

def shape_to_np(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = np.zeros((shape.num_parts, 2), dtype=dtype)

	# loop over all facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, shape.num_parts):
		coords[i] = (shape.part(i).x, shape.part(i).y)

	# return the list of (x, y)-coordinates
	return coords

def shape_to_list(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = []

	# loop over all facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, shape.num_parts):
		coords.append((shape.part(i).x, shape.part(i).y))

	# return the list of (x, y)-coordinates
	return coords

def calculateDistance(x1,y1,x2,y2):  
     dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
     return dist 

# calculates the orientation_sum for each picture
def calc_orientation_sums(pics, persons):
  # create list with base score 0 and same size as pics
  orientation_sums = []
  for i in range(len(pics)):
    orientation_sums.append(0)

  # add orientation of each face to the picture orientation score
  for person in persons:
    for face in person.faces:
      orientation_sums[face.picture_index] += face.score_orientation

  return orientation_sums

def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index
  
# filenames
landmarks_filename = "shape_predictor_68_face_landmarks.dat"
facerecognition_filename = "dlib_face_recognition_resnet_model_v1.dat"

# TODO: fill pictures with filenames
# upload pictures 
cv_pictures = [cv_img.copy() for cv_img in imgs]
dlib_pictures = [cv2.cvtColor(cv_img,cv2.COLOR_BGR2RGB) for cv_img in cv_pictures]

# init face detection and landmarks
face_detector = dlib.get_frontal_face_detector()
face_landmarks = dlib.shape_predictor(landmarks_filename)
face_recognition = dlib.face_recognition_model_v1(facerecognition_filename)
face_cascade_smile = cv2.CascadeClassifier('haarcascade_smile.xml')

# list of every unique person
persons = []

# iterate each picture and search for faces
for pic_idx, picture in enumerate(dlib_pictures):
  # detect faces in the image
  faces = face_detector(picture, 0)

  # TODO: evlt Schleife angucken, was genau ist d?
  # d is the rectangle of the face
  # get landmarks of each face
  # get id for each face
  for k,d in enumerate(faces):
    roi = extractFace(picture, d)
    height, width, notneeded = roi.shape
    landmarks = face_landmarks(roi,dlib.rectangle(0,0,width,height))
    face_id = face_recognition.compute_face_descriptor(roi,landmarks)

    # check if person is already in persons list
    person = next((person for person in persons if person.faceVectorMatches(face_id)), None)
    if person is None:
      person = Person(face_id)
      persons.append(person)

    face = Face(roi, pic_idx, landmarks,d)
    face.calc_scores()
    face.calc_face_mask()
    person.faces.append(face) 
    
# Output plots
for person in persons:
  print(len(person.faces))
  person.normalize_scores()
  person.faces.sort(key=lambda face:face.score_all,reverse=True)

  fig, ax = plt.subplots(nrows=1,ncols=len(person.faces),figsize=(15,15))
  for i,a in enumerate(ax):
    a.imshow(person.faces[i].picture_data)
    a.set_title(person.faces[i].scores_str())
    a.axis("off")

  plt.show()
  print("---------")

# get orientation sums, one sum for each pic
orientation_sums = calc_orientation_sums(cv_pictures,persons)
# sort dlib pictures based on these scores
sorted_pics = [cv_pictures for _, cv_pictures in sorted(zip(orientation_sums,cv_pictures), key=lambda pair: pair[0],reverse=True)]
# sort orientation sums
orientation_sums.sort(reverse=True)
# output picture list, only needed for debugging
if False == True:
  fig, ax = plt.subplots(nrows=1,ncols=len(sorted_pics),figsize=(100,100))
  for i,a in enumerate(ax):
    a.imshow(sorted_pics[i])
    a.set_title(orientation_sums[i])
    a.axis("off")
  plt.show()

# first picture of sorted_pics is the one with the highest orientation score, because its sorted 
base_picture = sorted_pics[0]
height, width, notneeded = base_picture.shape
cv2_imshow(cv2.resize(base_picture,(int(width/4),int(height/4))))

# face swap

# get dst_pic_index
dst_pic_index = None
for idx, pic in enumerate(cv_pictures):
  if pic is base_picture:
    dst_pic_index =  idx

for person in persons:
  # get src face
  src_face = person.faces[0]
  #src_face_img = cv2.cvtColor(src_face.picture_data, cv2.COLOR_BGR2RGB)
  print("SRC FACE:")
  cv2_imshow(cv2.cvtColor(src_face.picture_data,cv2.COLOR_RGB2BGR))

  # get dst_face
  dst_face_test = person.getFaceFromPic(dst_pic_index)
  dst_face = base_picture
  #dst_face_img = cv2.cvtColor(dst_face.picture_data, cv2.COLOR_BGR2RGB)
  #print("DST FACE:")
  #cv2_imshow(cv2.cvtColor(dst_face.picture_data,cv2.COLOR_RGB2BGR))

  if src_face.picture_data is dst_face_test.picture_data:
    print("its the same face :(")
    continue

  src_face.delauney_triangulation()
  triangles = src_face.triangles

  landmarks1 = shape_to_np(src_face.landmarks,dtype=np.int32)
  #landmarks2 = shape_to_np(dst_face.landmarks,dtype=np.int32)
  landmarks2 = dst_face_test.landmarksToFullSize()

  hullTest = cv2.convexHull(landmarks2)
  cv2.fillConvexPoly(dst_face,hullTest,(255,0,0))
  cv2_imshow(dst_face)

  indexes_triangles = []
  for t in triangles:
    pt1 = (t[0], t[1])
    pt2 = (t[2], t[3])
    pt3 = (t[4], t[5])

    index_pt1 = np.where((landmarks1 == pt1).all(axis=1))
    index_pt1 = extract_index_nparray(index_pt1)

    index_pt2 = np.where((landmarks1 == pt2).all(axis=1))
    index_pt2 = extract_index_nparray(index_pt2)

    index_pt3 = np.where((landmarks1 == pt3).all(axis=1))
    index_pt3 = extract_index_nparray(index_pt3)

    if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
        triangle = [index_pt1, index_pt2, index_pt3]
        indexes_triangles.append(triangle)
  
  t1 = []
  t2 = []
  #get points for img1, img2 corresponding to the triangles
  for triangle_index in indexes_triangles:
    t1_p1 = landmarks1[triangle_index[0]]
    t1_p2 = landmarks1[triangle_index[1]]
    t1_p3 = landmarks1[triangle_index[2]]
    triangle1 = np.array([t1_p1,t1_p2,t1_p3])

    t2_p1 = landmarks2[triangle_index[0]]
    t2_p2 = landmarks2[triangle_index[1]]
    t2_p3 = landmarks2[triangle_index[2]]
    triangle2 = np.array([t2_p1,t2_p2,t2_p3])

    t1.append(triangle1)
    t2.append(triangle2)

  for i in range(0,len(indexes_triangles)):
    warpTriangle(src_face.picture_data, dst_face, t1[i], t2[i])

  # Calculate Mask

  mask = dst_face_test.mask

  #cv2.fillConvexPoly(mask, np.int32(mask), (255, 255, 255))
  
  hull2 = landmarks2[:22]
  hull2 = cv2.convexHull(hullArray)

  #r = cv2.boundingRect(np.float32([hull2]))    
  x = dst_face_test.coords.left()
  y = dst_face_test.coords.top()
  w =dst_face_test.coords.right()-x
  h=dst_face_test.coords.bottom()-y
  #r = cv2.boundingRect([(x,y),(w,h)])
  center = ((x+int(w/2), y+int(h/2)))
  #center = (int((x+x+w)/2),int((y+y+h)/2))    

  # Clone seamlessly.
  output = cv2.seamlessClone(np.uint8(dst_face_test.picture_data), base_picture, mask, center, cv2.NORMAL_CLONE)
  base_picture = output
  print("DURCHLAUF NEU ----------------")

print("FINAL PICTURE:")
width, height, _ = base_picture.shape
cv2_imshow(base_picture)